In [1]:
# from tensorflow.examples.tutorials.mnist import input_data

# def mnist_data_iteratior():
#     mnist = input_data.read_data_sets('./newdata', one_hot=True)
#     def iterator(hparams, num_batches):
#         for _ in range(num_batches):
#             yield mnist.train.next_batch(hparams.batch_size)
#     return iterator

In [2]:
# mnist_data_iteratior()


In [3]:
def encoder(hparams, x_ph, scope_name, reuse):
    with tf.variable_scope(scope_name) as scope:
        if reuse:
            scope.reuse_variables()

        w1 = tf.get_variable('w1', initializer=xavier_init(hparams.n_input, hparams.n_hidden_recog_1))
        b1 = tf.get_variable('b1', initializer=tf.zeros([hparams.n_hidden_recog_1], dtype=tf.float32))
        hidden1 = hparams.transfer_fct(tf.matmul(x_ph, w1) + b1)

        w2 = tf.get_variable('w2', initializer=xavier_init(hparams.n_hidden_recog_1, hparams.n_hidden_recog_2))
        b2 = tf.get_variable('b2', initializer=tf.zeros([hparams.n_hidden_recog_2], dtype=tf.float32))
        hidden2 = hparams.transfer_fct(tf.matmul(hidden1, w2) + b2)

        w3 = tf.get_variable('w3', initializer=xavier_init(hparams.n_hidden_recog_2, hparams.n_z))
        b3 = tf.get_variable('b3', initializer=tf.zeros([hparams.n_z], dtype=tf.float32))
        z_mean = tf.matmul(hidden2, w3) + b3

        w4 = tf.get_variable('w4', initializer=xavier_init(hparams.n_hidden_recog_2, hparams.n_z))
        b4 = tf.get_variable('b4', initializer=tf.zeros([hparams.n_z], dtype=tf.float32))
        z_log_sigma_sq = tf.matmul(hidden2, w4) + b4

    return z_mean, z_log_sigma_sq


def generator(hparams, z, scope_name, reuse):

    with tf.variable_scope(scope_name) as scope:
        if reuse:
            scope.reuse_variables()

        w1 = tf.get_variable('w1', initializer=xavier_init(hparams.n_z, hparams.n_hidden_gener_1))
        b1 = tf.get_variable('b1', initializer=tf.zeros([hparams.n_hidden_gener_1], dtype=tf.float32))
        hidden1 = hparams.transfer_fct(tf.matmul(z, w1) + b1)

        w2 = tf.get_variable('w2', initializer=xavier_init(hparams.n_hidden_gener_1, hparams.n_hidden_gener_2))
        b2 = tf.get_variable('b2', initializer=tf.zeros([hparams.n_hidden_gener_2], dtype=tf.float32))
        hidden2 = hparams.transfer_fct(tf.matmul(hidden1, w2) + b2)

        w3 = tf.get_variable('w3', initializer=xavier_init(hparams.n_hidden_gener_2, hparams.n_input))
        b3 = tf.get_variable('b3', initializer=tf.zeros([hparams.n_input], dtype=tf.float32))
        logits = tf.matmul(hidden2, w3) + b3
        x_reconstr_mean = tf.nn.sigmoid(logits)

    return logits, x_reconstr_mean


def get_loss(x, logits, z_mean, z_log_sigma_sq):
    reconstr_losses = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=logits), 1)
    latent_losses = -0.5 * tf.reduce_sum(1 + z_log_sigma_sq - tf.square(z_mean) - tf.exp(z_log_sigma_sq), 1)
    total_loss = tf.reduce_mean(reconstr_losses + latent_losses, name='total_loss')
    return total_loss


def get_z_var(hparams, batch_size):
    z = tf.Variable(tf.random_normal((batch_size, hparams.n_z)), name='z')
    return z


def gen_restore_vars():
    restore_vars = ['gen/w1',
                    'gen/b1',
                    'gen/w2',
                    'gen/b2',
                    'gen/w3',
                    'gen/b3']
    return restore_vars

In [4]:
def save_images(images, size, image_path):
    print('----------------------------------saveeeeeeeeeeeeee---------',images.shape)
    print(size)
    return imsave(inverse_transform(images), size, image_path)


def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))


def inverse_transform(images):
    return (images+1.)/2


def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image
    return img


def print_hparams(hparams):
    print ('')
    for temp in dir(hparams):
        if temp[:1] != '_':
            print ('{0} = {1}'.format(temp, getattr(hparams, temp)))
    print ('')


def set_up_dir(directory, clean=False):
    if os.path.exists(directory):
        if clean:
            shutil.rmtree(directory)
    else:
        os.makedirs(directory)


def get_ckpt_path(ckpt_dir):
    ckpt_dir = os.path.abspath(ckpt_dir)
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_path = os.path.join(ckpt_dir,
                                 ckpt.model_checkpoint_path)
    else:
        ckpt_path = None
    return ckpt_path


def try_restore(hparams, sess, model_saver):
    """Attempt to restore variables from checkpoint"""
    ckpt_path = get_ckpt_path(hparams.ckpt_dir)
    if ckpt_path:  # if a previous ckpt exists
        model_saver.restore(sess, ckpt_path)
        start_epoch = int(ckpt_path.split('/')[-1].split('-')[-1])
        print ('Succesfully loaded model from {0} at counter = {1}'.format(
            ckpt_path, start_epoch))
    else:
        print ('No checkpoint found')
        start_epoch = -1
    return start_epoch



def xavier_init(fan_in, fan_out, constant=1):
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0 / (fan_in + fan_out))
    high = constant*np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)

In [5]:
class Hparams(object):
    def __init__(self):
        self.n_hidden_recog_1 = 500  # 1st layer encoder neurons
        self.n_hidden_recog_2 = 500  # 2nd layer encoder neurons
        self.n_hidden_gener_1 = 500  # 1st layer decoder neurons
        self.n_hidden_gener_2 = 500  # 2nd layer decoder neurons
        self.n_input = 784           # MNIST data input (img shape: 28*28)
        self.n_z = 20                # dimensionality of latent space
        self.transfer_fct = tf.nn.softplus


In [6]:
def load_data(directory_path):
    print(len(os.listdir(directory_path)))
    train_data = []
    for file_ in sorted(os.listdir(directory_path)):
        if file_.endswith('.png'):
            if directory_path.endswith('/'):
                 image_path = directory_path + file_
            else: image_path = directory_path + '/' + file_

            image = imread(image_path)/255.0 # Normalize values
            image = np.resize(image, (28,28))# added pankaj
            image = np.expand_dims(image, axis=-1) # Add channel dim
            train_data.append(image)
    train_data = np.array(train_data)
    return train_data

In [7]:
# data = load_data('resized/')[:3900]
# data.shape

In [8]:
# final_data = []

# for i in range(39):
#     final_data.append(data[(0+i)*100:(1+i)*100])

# final_data = np.array(final_data)

# final_data[0].shape

# final_list = []
# final_list = [data[(0+i)*100:(1+i)*100] for i in range(39)]
# final_data = np.array(final_list)

In [9]:
# final_data

In [10]:
import os
import numpy as np
import tensorflow as tf
import scipy
from scipy.ndimage import imread
from scipy.misc import imsave


def main(hparams):
    # Set up some stuff according to hparams
    set_up_dir(hparams.ckpt_dir)
    set_up_dir(hparams.sample_dir)
    print_hparams(hparams)

    # encode
    x_ph = tf.placeholder(tf.float32, [None, hparams.n_input], name='x_ph')
    z_mean, z_log_sigma_sq = encoder(hparams, x_ph, 'enc', reuse=False)

    # sample
    eps = tf.random_normal((hparams.batch_size, hparams.n_z), 0, 1, dtype=tf.float32)
    z_sigma = tf.sqrt(tf.exp(z_log_sigma_sq))
    z = z_mean + z_sigma * eps

    # reconstruct
    logits, x_reconstr_mean = generator(hparams, z, 'gen', reuse=False)

    # generator sampler
    z_ph = tf.placeholder(tf.float32, [None, hparams.n_z], name='x_ph')
    _, x_sample = generator(hparams, z_ph, 'gen', reuse=True)

    # define loss and update op
    total_loss = get_loss(x_ph, logits, z_mean, z_log_sigma_sq)
    opt = tf.train.AdamOptimizer(learning_rate=hparams.learning_rate)
    update_op = opt.minimize(total_loss)

    # Sanity checks
    for var in tf.global_variables():
        print (var.op.name)
    print ('')

    # Get a new session
    sess = tf.Session()

    # Model checkpointing setup
    model_saver = tf.train.Saver()

    init_op = tf.global_variables_initializer()
    sess.run(init_op)

    # Attempt to restore variables from checkpoint
    start_epoch = try_restore(hparams, sess, model_saver)

    # Get data iterator
    #iterator = mnist_data_iteratior()
    #print('shape of x_ph', x_ph.shape)

    
    data = load_data('resized/')[:3900]
    
    final_data = []

    for i in range(39):
        final_data.append(data[(0+i)*100:(1+i)*100])

    final_data = np.array(final_data)
    print('---------after data--------')
    
    
    # Training
    for epoch in range(start_epoch+1, hparams.training_epochs):
        avg_loss = 0.0
        num_batches = hparams.num_samples // hparams.batch_size
        batch_num = 0
        for x_batch_val in final_data:
        #for (x_batch_val,_) in load_data('resized/'):
            print('batchhhhhhh---------------',x_batch_val.shape)
            batch_num += 1
            #print('in for loop --------', x_batch_val)
            
#             print('-------------',type(x_batch_val))
            x_batch_val = np.reshape(x_batch_val, (100, 28*28))
#            print('size---------------',x_batch_val.shape)
            
            feed_dict = {x_ph: x_batch_val}
            _, loss_val = sess.run([update_op, total_loss], feed_dict=feed_dict)
            avg_loss += loss_val / hparams.num_samples * hparams.batch_size
            print('average loss', avg_loss)
            if batch_num % 100 == 0:
                x_reconstr_mean_val = sess.run(x_reconstr_mean, feed_dict={x_ph: x_batch_val})

                z_val = np.random.randn(hparams.batch_size, hparams.n_z)
                x_sample_val = sess.run(x_sample, feed_dict={z_ph: z_val})
                
                print('------------before saving-------------')
                print(x_reconstr_mean_val.shape)

                save_images(np.reshape(x_reconstr_mean_val, [-1, 28, 28]),
                                  [10,10],
                                  '{}/reconstr_{:02d}_{:04d}.png'.format(hparams.sample_dir, epoch, batch_num))
                print('------------after saving-------------')
                save_images(np.reshape(x_batch_val, [-1, 28, 28]),
                                  [10,10],
                                  '{}/orig_{:02d}_{:04d}.png'.format(hparams.sample_dir, epoch, batch_num))
                save_images(np.reshape(x_sample_val, [-1, 28, 28]),
                                  [10,10],
                                  '{}/sampled_{:02d}_{:04d}.png'.format(hparams.sample_dir, epoch, batch_num))


        if epoch % hparams.summary_epoch == 0:
            print ("Epoch:", '%04d' % (epoch), 'Avg loss = {:.9f}'.format(avg_loss))

        if epoch % hparams.ckpt_epoch == 0:
            save_path = os.path.join(hparams.ckpt_dir, 'mnist_vae_model')
            model_saver.save(sess, save_path, global_step=epoch)

    save_path = os.path.join(hparams.ckpt_dir, 'mnist_vae_model')
    model_saver.save(sess, save_path, global_step=hparams.training_epochs-1)


if __name__ == '__main__':

    HPARAMS = Hparams()

    HPARAMS.num_samples = 60000
    HPARAMS.learning_rate = 0.001
    HPARAMS.batch_size = 100
    HPARAMS.training_epochs = 100
    HPARAMS.summary_epoch = 1
    HPARAMS.ckpt_epoch = 5

    HPARAMS.ckpt_dir = './models/mnist-vae/'
    HPARAMS.sample_dir = './samples/mnist-vae/'

    main(HPARAMS)

/Users/jitu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



batch_size = 100
ckpt_dir = ./models/mnist-vae/
ckpt_epoch = 5
learning_rate = 0.001
n_hidden_gener_1 = 500
n_hidden_gener_2 = 500
n_hidden_recog_1 = 500
n_hidden_recog_2 = 500
n_input = 784
n_z = 20
num_samples = 60000
sample_dir = ./samples/mnist-vae/
summary_epoch = 1
training_epochs = 100
transfer_fct = <function softplus at 0x111b21ae8>

enc/w1
enc/b1
enc/w2
enc/b2
enc/w3
enc/b3
enc/w4
enc/b4
gen/w1
gen/b1
gen/w2
gen/b2
gen/w3
gen/b3
beta1_power
beta2_power
enc/w1/Adam
enc/w1/Adam_1
enc/b1/Adam
enc/b1/Adam_1
enc/w2/Adam
enc/w2/Adam_1
enc/b2/Adam
enc/b2/Adam_1
enc/w3/Adam
enc/w3/Adam_1
enc/b3/Adam
enc/b3/Adam_1
enc/w4/Adam
enc/w4/Adam_1
enc/b4/Adam
enc/b4/Adam_1
gen/w1/Adam
gen/w1/Adam_1
gen/b1/Adam
gen/b1/Adam_1
gen/w2/Adam
gen/w2/Adam_1
gen/b2/Adam
gen/b2/Adam_1
gen/w3/Adam
gen/w3/Adam_1
gen/b3/Adam
gen/b3/Adam_1

INFO:tensorflow:Restoring parameters from /Users/jitu/Documents/Fall/Business Data Science/finalproject/models/mnist-vae/mnist_vae_model-99
Succesfully loaded model fr

/Users/jitu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.


---------after data--------


In [11]:
# final_data

In [12]:
# import os
# from PIL import Image
# from array import *
# from random import shuffle

# # Load from and save to
# Names = [['./resized','train']]

# for name in Names:
	
# 	data_image = array('B')
# 	data_label = array('B')

# 	FileList = []
# 	for dirname in os.listdir(name[0])[1:]: # [1:] Excludes .DS_Store from Mac OS
# 		path = os.path.join(name[0],dirname)
# 		for filename in os.listdir(path):
# 			if filename.endswith(".png"):
# 				FileList.append(os.path.join(name[0],dirname,filename))

# 	shuffle(FileList) # Usefull for further segmenting the validation set

# 	for filename in FileList:

# 		label = int(filename.split('/')[2])

# 		Im = Image.open(filename)

# 		pixel = Im.load()

# 		width, height = Im.size

# 		for x in range(0,width):
# 			for y in range(0,height):
# 				data_image.append(pixel[y,x])

# 		data_label.append(label) # labels start (one unsigned byte each)

# 	hexval = "{0:#0{1}x}".format(len(FileList),6) # number of files in HEX

# 	# header for label array

# 	header = array('B')
# 	header.extend([0,0,8,1,0,0])
# 	header.append(int('0x'+hexval[2:][:2],16))
# 	header.append(int('0x'+hexval[2:][2:],16))
	
# 	data_label = header + data_label

# 	# additional header for images array
	
# 	if max([width,height]) <= 256:
# 		header.extend([0,0,0,width,0,0,0,height])
# 	else:
# 		raise ValueError('Image exceeds maximum size: 256x256 pixels');

# 	header[3] = 3 # Changing MSB for image data (0x00000803)
	
# 	data_image = header + data_image

# 	output_file = open(name[1]+'-images-idx3-ubyte', 'wb')
# 	data_image.tofile(output_file)
# 	output_file.close()

# 	output_file = open(name[1]+'-labels-idx1-ubyte', 'wb')
# 	data_label.tofile(output_file)
# 	output_file.close()

# # gzip resulting files

# for name in Names:
# 	os.system('gzip '+name[1]+'-images-idx3-ubyte')
# 	os.system('gzip '+name[1]+'-labels-idx1-ubyte')